In [7]:
import os
import glob
import pandas as pd
import textgrids
import tqdm
from utils import flac_to_wav

# Create Subset json

## Keep files in subset

In [31]:
import ast
import pandas as pd

def genorate_pps_df(json_dir:str):
    with open(json_dir, 'r') as json_file:
        json_list = list(json_file)

    df = pd.DataFrame.from_dict(json_list)
    df[0] = df[0].apply(lambda x: ast.literal_eval(x))

    for keys in df[0][0].keys():
        df[keys] = [path[0][keys] for path in df.iloc()]
    df = df.drop(0, axis=1)

    df = df[['training_data']] # Keep only columbs that are needed to split
    for keys in df['training_data'][0].keys():
        df[keys] = [path[0][keys] for path in df.iloc()]
    df = df[['label', 'name']] # Keep only columbs that are needed to split
    #audio_path, duration, shard_id, text

    subset_df = pd.DataFrame()

    labels = []
    names = []

    for row in df.iloc():
        label, name = row
        for l, n in zip(label, name):
            labels.append(l)
            names.append(n)
    
    subset_df['audio_path'] = names
    subset_df['text'] = labels

    return subset_df

def get_subset_df(dataset_root_path:str, df:pd.DataFrame):
    subset = glob.glob(dataset_root_path, recursive=True)
    subset = [os.path.join(*(dir.split(os.path.sep)[5:])) for dir in subset]

    subset_df = df[df['name'].isin(subset)].reset_index(drop=True)
    subset_df = subset_df.drop(0, axis=1)

    return subset_df


df = genorate_pps_df('/home/knoriy/split_peoples_speech/subset.json')
# df.to_csv('/home/knoriy/Documents/laion/split_peoples_speech/mini_subset.tsv', sep='\t', header=None, index=False)
df

,audio_path,text
0,CC20160404BudgetFinance2of2/CC-2016-0404-Budge...,members let us return to order that the it is ...
1,CC20160404BudgetFinance2of2/CC-2016-0404-Budge...,would ask mr buzz if he has any comments to sh...
2,CC20160404BudgetFinance2of2/CC-2016-0404-Budge...,its community development block grant program ...
3,CC20160404BudgetFinance2of2/CC-2016-0404-Budge...,patient our early on the projects that we've s...
4,CC20160404BudgetFinance2of2/CC-2016-0404-Budge...,thank you very much that's all chair thank yo...
...,...,...
8469,Sleepfor1950/Sleepfor1950_DOT_mp3_00027.flac,you don't have much fun when you're tired and ...
8470,Sleepfor1950/Sleepfor1950_DOT_mp3_00028.flac,you're irritable and cross hard to get along with
8471,Sleepfor1950/Sleepfor1950_DOT_mp3_00029.flac,people who don't like you and you don't like y...
8472,Sleepfor1950/Sleepfor1950_DOT_mp3_00030.flac,each night makes it easier for you to go right...


# prepare MFA

In [32]:
# Clean old wavs ONLY FOR TESTING
import glob

old_wav_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/mini_subset/**/*.wav', recursive=True)
for wav_path in old_wav_path:
    os.remove(wav_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset_split/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# Tar loader

# ToDo
- [ ] Download and extract single or batch of files from aws
- [ ] Process for MFA
- [ ] Generate textgrid
- [ ] Split audio and upload to aws s3

# AWS S3

In [ ]:
%%bash

nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-07a8f0d3-6d27-4299-887a-dc12a6d72f8d-c000.tar?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A30%3A22Z/-1/host/444b9c082ceffd10f38bb965679ed9ec12202836831e111dd193fde281062d26 -O - | aws s3 cp - s3://s-laion/peoples_speech/train_clean_pps.tar" &;
nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-4e132642-c01c-4db6-9db0-a1e19193f6f8-c000.json?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A31%3A22Z/-1/host/d7dacf3c31d2e3670d82727636ce234be27a9128df7a80883b84b4a3d8c7f6c0 -O - | aws s3 cp - s3://s-laion/peoples_speech/Manifest.json" &;


In [ ]:
%%bash

nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [ ]:
%%bash

# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [30]:
import tarfile

metadata_dir = "/mnt/knoriy/metadata.json"
tar_dir = "/mnt/knoriy/pps_train.tar"

with tarfile.open(tar_dir, mode='r') as src_file_obj:
    print('opening file: This may take some time\n')

    file_names_full_list = src_file_obj.getnames()
    file_names_full_list = [i for i in file_names_full_list if '.flac' in i]

opening file: This may take some time



In [31]:
file_names_full_list[0]

'CC20160404BudgetFinance2of2/CC-2016-0404-BudgetFinance_2of2_DOT_mp3_00000.flac'